# Madagascar 2 - DroneMDT (Metadata) Notebook

### Common variables:

In [27]:
# Script folder
dronemdt_folder = '/home/sylvain/DroneMDT'
# Session / Flight data input
input = '/home/sylvain/HDD5To/DATA/Mada/julien'
# Output geoflow config (json and csv entities)
output = '/home/sylvain/HDD5To/DATA/Mada/julien'
# Geoflow folder where .env is stored and jobs will be available
geoflow_folder = '/home/sylvain/geoflow-g2oi'
# Geoserver configuration
WS = 'IHSM'
DS = 'dronemadagascar'
# Zenodo setup
sandbox = False
# Move to dronemdt_folder
import os
os.chdir(dronemdt_folder)

## Part 1a: Check images validity

In [104]:
# DATA have to be copied from Drone internal storage or sdcard DCIM folder using suh naming convention:
# YYYYMMDD_COUNTRYCODE-optionalplace_device_session-number
# ├── DCIM :  folder to store videos and photos depending on the media collected
#
# Check validity of images copied from Drone to disk
# rclone copy cloudIRD:/G2OI/Missions/Aldabra /media/HDD5To/DATA/ALDABRA/

In [105]:
from check_images import *
img_list = im_list(input)
for i in range(len(img_list)):
    check_image_with_pil(img_list[i])

In [106]:
from check_images import *
img_list = im_list(input)
for i in range(len(img_list)):
    # print(i, img_list[i])
    imgv = Image.open(img_list[i])
    # imgv = check_image_with_pil(img_list[i])
    print(img_list[i], imgv)

/home/sylvain/HDD5To/DATA/MADAGASCAR/20230505_MDG-sarodrano_UAV-02_2/00_sample_rawdata_overview.png <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=960x720 at 0x7F70A01C7430>
/home/sylvain/HDD5To/DATA/MADAGASCAR/20230505_MDG-sarodrano_UAV-02_2/PROCESSED_DATA/PHOTOGRAMMETRY/odm_texturing/odm_textured_model_geo_material0010_map_Kd.png <PIL.PngImagePlugin.PngImageFile image mode=RGB size=8192x8192 at 0x7F6DEF5FA4D0>
/home/sylvain/HDD5To/DATA/MADAGASCAR/20230505_MDG-sarodrano_UAV-02_2/PROCESSED_DATA/PHOTOGRAMMETRY/odm_texturing/odm_textured_model_geo_material0020_map_Kd.png <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7F87C69B5900>
/home/sylvain/HDD5To/DATA/MADAGASCAR/20230505_MDG-sarodrano_UAV-02_2/PROCESSED_DATA/PHOTOGRAMMETRY/odm_texturing/odm_textured_model_geo_material0011_map_Kd.png <PIL.PngImagePlugin.PngImageFile image mode=RGB size=8192x8192 at 0x7F6DEF5FA4D0>
/home/sylvain/HDD5To/DATA/MADAGASCAR/20230505_MDG-sarodrano_UAV-02_2/PROCESSED_DATA/PHOTOGRAM

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Part 1b: Generate report

In [26]:
## Using subprocess:
# import subprocess
# subprocess.check_output(["Rscript", "/home/sylvain/DroneMDT/generate_drone_metadata.R", input])
# from rpy2.robjects.packages import importr


Loading required package: magrittr
uasimg (version 1.8.3)
Bug reports: https://github.com/ucanr-igis/uasimg/issues
/home/sylvain/HDD5To/DATA/Mada/julien/20230427_MDG-Toliara-Mangroves_UAV-01_1/DCIM
 - Looking for image files (all image files)
Error in uas_info(file.path(inputdir, "DCIM"), fp = TRUE, cache = TRUE,  : 
  Couldn't find any image files that match the pattern in /home/sylvain/HDD5To/DATA/Mada/julien/20230427_MDG-Toliara-Mangroves_UAV-01_1/DCIM
Execution halted


CalledProcessError: Command '['Rscript', '/home/sylvain/DroneMDT/generate_drone_metadata.R', '/home/sylvain/HDD5To/DATA/Mada/julien']' returned non-zero exit status 1.

## Part 2: Generate metadata

In [56]:
# Reload modules if they change on disk
%load_ext autoreload
%autoreload 2
# Import packages and modules
import os
import shutil
import pandas as pd
import geopandas as gpd
import datetime
# from IPython.display import display
import argparse
import json
import pycountry
os.chdir(dronemdt_folder)
from meteo_helper import meteo
from read_metadata import *
from exif_helper import *
from enriched_func import *
from zenodo_helper import *
from zip_helper import *
from geoflow_helper import *
from raster_helper import *
from process_webodm import *
from settings import *
import hashlib
from report import *
import pdfkit
# import wkhtmltopdf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
### Filter foder name
folder_name = filter_folder_name(input, 'UAV')
print("Number of folder: ", len(folder_name))
folder_name

Number of folder:  34


['20230430_MDG-Nosy-Ve_UAV-01_6',
 '20230501_MDG-Anakao-Prince-Anakao_UAV-01_4',
 '20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_14',
 '20230501_MDG-Anakao-Prince-Anakao_UAV-01_7',
 '20230507_MDG-Mangily-estran_UAV-01_3',
 '20230502_MDG-Sarodrano-IOT_UAV-01_5',
 '20230430_MDG-Nosy-Ve_UAV-01_5',
 '20230505_MDG-Sarodrano-village_UAV-01_1',
 '20230504_MDG-GRT-micro-atoll-zone-B_UAV-01_1',
 '20230503_MDG-Sarodrano-IOT_UAV-01_11',
 '20230502_MDG-Sarodrano-IOT_UAV-01_6',
 '20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_15',
 '20230501_MDG-Anakao-Prince-Anakao_UAV-01_6',
 '20230430_MDG-Nosy-Ve_UAV-01_3',
 '20230506_MDG-Mangily-estran_UAV-01_2',
 '20230505_MDG-GRT-fausse-passe_UAV-01_7',
 '20230502_MDG-Sarodrano-pirogues_UAV-01_7',
 '20230427_MDG-Toliara-Mangroves_UAV-01_1',
 '20230507_MDG-Mangily-estran_UAV-01_4',
 '20230430_MDG-Nosy-Ve_UAV-01_2',
 '20230504_MDG-Nosy-Velomitahy_UAV-01_6',
 '20230507_MDG-Mangily-Massif-Roses_UAV-01_3',
 '20230430_MDG-Nosy-Ve_UAV-01_1',
 '20230504_MDG-GR

In [49]:
### Put data in basic panda dataframe
df = {}
df = basic_geoflow_df(folder_name)
df.head()

,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,20230430_MDG-Nosy-Ve_UAV-01_6,title:20230430 mdg nosy ve uav 01 6,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-11-21_\nedition:2023-11-21,dataset,eng,,MDG-Nosy-Ve,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None
1,20230501_MDG-Anakao-Prince-Anakao_UAV-01_4,title:20230501 mdg anakao prince anakao uav 01 4,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-11-21_\nedition:2023-11-21,dataset,eng,,MDG-Nosy-Ve,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None
2,20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_14,title:20230505 mdg sarodrano reef manjaka gauche uav 01 14,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-11-21_\nedition:2023-11-21,dataset,eng,,MDG-Nosy-Ve,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None
3,20230501_MDG-Anakao-Prince-Anakao_UAV-01_7,title:20230501 mdg anakao prince anakao uav 01 7,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-11-21_\nedition:2023-11-21,dataset,eng,,MDG-Nosy-Ve,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None
4,20230507_MDG-Mangily-estran_UAV-01_3,title:20230507 mdg mangily estran uav 01 3,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-11-21_\nedition:2023-11-21,dataset,eng,,MDG-Nosy-Ve,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None


In [50]:
### Define Metadata file
md_file = 'metadata.txt'
### Parse global metadata
mdt_df_glob = global_mdt(input, md_file)
#### Enrich metadata
enrich_drone_mdt(input, df, md_file, mdt_df_glob)

INFO: Global `metadata.txt` found !
WARN: No metadata.txt in /home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-01_6/METADATA to parse
INFO: GPS Folder found
Format id to obtain date, country name, optionnal place, and platform
####################
### 2. Enrich Title
 ####################
INFO: Using glocal title
Images drone {platform}, {place}, {country} - {date}
format field
####################
### 8. Enrich Language
 ####################
INFO: Using glocal language
####################
### 4. Enrich Subject
 ####################
INFO: Using glocal theme
INFO: Using glocal project
####################
### 5. Enrich Creator
 ####################
INFO: Using glocal author
INFO: Using glocal publisher
INFO: Using glocal owner
INFO: Using glocal pointOfContact
INFO: Using glocal processor
####################
### 11. Enrich Relation
 ####################
INFO: Using glocal thumbnail
INFO: Using glocal http
####################
### Reading DCIM folder
 ####################


/home/sylvain/DroneMDT/enriched_func.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  enriched = df_global.iloc[0][keyword][0]


/home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-01_6
Generate image with a set of images
Nb images:  42
preview 8 x  5
####################
### 10. Enrich TemporalCoverage
 ####################
####################
### 00. Prepare Provenance (13) and Description (3) 
 ####################
/home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-01_6/DCIM/102MEDIA/DJI_0633.JPG
####################
Fetching open-meteo
 ####################
Location: -23.6440304, 43.600762950000004
Begin: 2023-04-30 14-06-14 - End: 2023-04-30 14-11-01
Model: best_match


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1988    0  1988    0     0   8211      0 --:--:-- --:--:-- --:--:--  8248


Convert Dataframe to UTM
Index(['FileName', 'GPSLatitude', 'GPSLongitude', 'GPSAltitude',
       'FlightRollDegree', 'FlightPitchDegree', 'FlightYawDegree',
       'GimbalRollDegree', 'GimbalPitchDegree', 'GimbalYawDegree',
       'ImageWidth', 'ImageLength', 'BitsPerSample', 'ImageDescription',
       'Make', 'Model', 'Orientation', 'SamplesPerPixel', 'XResolution',
       'YResolution', 'ResolutionUnit', 'Software', 'DateTime',
       'YCbCrPositioning', 'ExifOffset', 'GPSVersionID', 'ThumbnailImageWidth',
       'ThumbnailImageLength', 'ThumbnailCompression', 'ThumbnailXResolution',
       'ThumbnailYResolution', 'ThumbnailResolutionUnit',
       'ThumbnailJPEGInterchangeFormat',
       'ThumbnailJPEGInterchangeFormatLength', 'ExposureTime', 'FNumber',
       'ExposureProgram', 'ISOSpeedRatings', 'ExifVersion', 'DateTimeOriginal',
       'DateTimeDigitized', 'ComponentsConfiguration', 'ExposureBiasValue',
       'MaxApertureValue', 'MeteringMode', 'LightSource', 'Flash',
       'Foc

...

Convert Dataframe to UTM
Index(['FileName', 'GPSLatitude', 'GPSLongitude', 'GPSAltitude',
       'FlightRollDegree', 'FlightPitchDegree', 'FlightYawDegree',
       'GimbalRollDegree', 'GimbalPitchDegree', 'GimbalYawDegree',
       'ImageWidth', 'ImageLength', 'BitsPerSample', 'ImageDescription',
       'Make', 'Model', 'Orientation', 'SamplesPerPixel', 'XResolution',
       'YResolution', 'ResolutionUnit', 'Software', 'DateTime',
       'YCbCrPositioning', 'ExifOffset', 'GPSVersionID', 'ThumbnailImageWidth',
       'ThumbnailImageLength', 'ThumbnailCompression', 'ThumbnailXResolution',
       'ThumbnailYResolution', 'ThumbnailResolutionUnit',
       'ThumbnailJPEGInterchangeFormat',
       'ThumbnailJPEGInterchangeFormatLength', 'ExposureTime', 'FNumber',
       'ExposureProgram', 'ISOSpeedRatings', 'ExifVersion', 'DateTimeOriginal',
       'DateTimeDigitized', 'ComponentsConfiguration', 'ExposureBiasValue',
       'MaxApertureValue', 'MeteringMode', 'LightSource', 'Flash',
       'Foc

/home/sylvain/DroneMDT/enriched_func.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  enriched = df_global.iloc[0][keyword][0]


/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2
Generate image with a set of images
Nb images:  253
preview 8 x  8
####################
### 10. Enrich TemporalCoverage
 ####################
####################
### 00. Prepare Provenance (13) and Description (3) 
 ####################
/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/DCIM/101MEDIA/DJI_0354.JPG
####################
Fetching open-meteo
 ####################
Location: -23.502710049999997, 43.7240419
Begin: 2023-05-03 10-54-43 - End: 2023-05-03 11-06-42
Model: best_match


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1997    0  1997    0     0   7595      0 --:--:-- --:--:-- --:--:--  7622


Convert Dataframe to UTM
Index(['FileName', 'GPSLatitude', 'GPSLongitude', 'GPSAltitude',
       'FlightRollDegree', 'FlightPitchDegree', 'FlightYawDegree',
       'GimbalRollDegree', 'GimbalPitchDegree', 'GimbalYawDegree',
       'ImageWidth', 'ImageLength', 'BitsPerSample', 'ImageDescription',
       'Make', 'Model', 'Orientation', 'SamplesPerPixel', 'XResolution',
       'YResolution', 'ResolutionUnit', 'Software', 'DateTime',
       'YCbCrPositioning', 'ExifOffset', 'GPSVersionID', 'ThumbnailImageWidth',
       'ThumbnailImageLength', 'ThumbnailCompression', 'ThumbnailXResolution',
       'ThumbnailYResolution', 'ThumbnailResolutionUnit',
       'ThumbnailJPEGInterchangeFormat',
       'ThumbnailJPEGInterchangeFormatLength', 'ExposureTime', 'FNumber',
       'ExposureProgram', 'ISOSpeedRatings', 'ExifVersion', 'DateTimeOriginal',
       'DateTimeDigitized', 'ComponentsConfiguration', 'ExposureBiasValue',
       'MaxApertureValue', 'MeteringMode', 'LightSource', 'Flash',
       'Foc

In [51]:
# Set prefix for generated json and csv entities
out_prefix = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
out_prefix

'20231121-164317'

In [52]:
# Visualize output
df.head()

,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,20230430_MDG-Nosy-Ve_UAV-01_6,"title:Images drone UAV, Nosy ve, Madagascar - 20230430 - 01_6","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Nosy v...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,,2023-04-30 14-06-14 - 2023-04-30 14-11-01,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:SurveyMetadata.gpkg@/home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-01_6/GPS/Su...
1,20230501_MDG-Anakao-Prince-Anakao_UAV-01_4,"title:Images drone UAV, Anakao prince anakao, Madagascar - 20230501 - 01_4","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Anakao...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,,2023-05-01 08-29-43 - 2023-05-01 08-43-12,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:SurveyMetadata.gpkg@/home/sylvain/HDD5To/DATA/Mada/julien/20230501_MDG-Anakao-Prince-Anakao_UA...
2,20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_14,"title:Images drone UAV, Sarodrano reef manjaka gauche, Madagascar - 20230505 - 01_14","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Sarodr...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,,2023-05-05 08-56-23 - 2023-05-05 09-01-04,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:SurveyMetadata.gpkg@/home/sylvain/HDD5To/DATA/Mada/julien/20230505_MDG-Sarodrano-Reef-Manjaka-...
3,20230501_MDG-Anakao-Prince-Anakao_UAV-01_7,"title:Images drone UAV, Anakao prince anakao, Madagascar - 20230501 - 01_7","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Anakao...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,,2023-05-01 17-51-24 - 2023-05-01 17-56-17,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du pro

In [53]:
# Visualize Description
print(df.iloc[0]['Description'])

abstract:"Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Nosy ve, Madagascar à la date suivante : 20230430.
<br />Les vols ont été réalisés en partenariat avec l'IH.SM dans le but de créer des modèles numériques d'élévations pour cartographier l'écosystème marin.
<br />
<br /><b>Le dépôt est composé des éléments suivants:</b>
<br />    - 00_: Planche d'aperçu des images
<br />    - DCIM.zip: Images brutes issues du drone
<br />    - GPS.zip: Geopackage contenant l’emprise du survol ainsi que la géolocalisation des images accompagnées de leurs miniatures dans la table d’attribut en base64
<br />    - METADATA.zip: Métadonnées au format ISO19115, Rapports avec miniatures des images de drone (dossier tb) et statistiques de vols.
<br />
<br /><b>Arborescence d'origine:</b>
<br />│  └─ 20230430_MDG-Nosy-Ve_UAV-01_6
<br />│--------  └─ DCIM
<br />│----------------  └─ 102MEDIA
<br />│--------  └─ GPS
<br />│--------  └─ LABEL
<br />│--------  └─ METADATA

In [54]:
##############
#### ISO 19115 config
##############
## Export to csv
df.to_csv(os.path.join(output, out_prefix + "_iso19115-metadata.csv"), quoting=1, quotechar='"', index=False)
## Generate json
iso19115config = {'profile': {'id': 'iso19115', 
                              'name': 'iso19115', 
                              'project': 'Generate iso19115 with Geoflow', 
                              'organization': 'IRD', 
                              'environment': {'file': '.env', 
                                              'hide_env_vars': ['MOTDEPASSE']}, 
                              'logos': ['https://en.ird.fr/sites/ird_fr/files/2019-08/logo_IRD_2016_BLOC_UK_COUL.png'], 
                              'mode': 'entity'}, 
                  'metadata': {'entities': 
                               [{'handler': 'csv', 
                                 'source': os.path.join(output, out_prefix) + "_iso19115-metadata.csv"}], 
                               'contacts': 
                               [{'handler': 'csv', 
                                 'source': 'https://drive.ird.fr/s/EYS3qccyB28PrA9/download/geoflow_g2oi_contacts.csv'}]}, 
                  'software': [], 
                  'actions': 
                  [{'id': 'geometa-create-iso-19115',
                    'options' : 
                    {"logo": True}, 
                    "run": True}]}

# Export json to file
with open(os.path.join(output, out_prefix + '_iso19115-config.json'), 'w') as f:
    json.dump(iso19115config, f, indent=4)

In [71]:
# Optional : could start from here => reload dataframe from csv
df = {}
df = pd.read_csv(os.path.join(output, out_prefix + "_iso19115-metadata.csv"))

In [57]:
### Execute geoflow
xml19115 = run_geoflow(os.path.join(output, out_prefix + '_iso19115-config.json'), geoflow_folder)
os.chdir(dronemdt_folder)

R[write to console]: Rows: 129 Columns: 14

R[write to console]: ── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (14): Identifier, Email, OrganizationName, PositionName, LastName, First...

R[write to console]: 
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

R[write to console]: Rows: 34 Columns: 15

R[write to console]: ── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (15): Identifier, Title, Description, Subject, Creator, Date, Type, Lang...

R[write to console]: 
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

R[write to console]: Loading required package: geometa

R[write to console]: Loading required package: ows4R



In [58]:
# Geoflow job log folder
print(xml19115[0])

/home/sylvain/geoflow-g2oi/jobs/20231121164441


In [59]:
## Copy generated ISO19115 to original metadata folder
for id in range(len(df)):
    jobs19115 = xml19115[0] + '/entities/' + df.iloc[id]['Identifier'] + '/metadata/'
    fname19115 = df.iloc[id]['Identifier'] + '_ISO-19115.xml'
    if fname19115 in os.listdir(jobs19115):
        shutil.copyfile(jobs19115+fname19115, 
                        input + "/" + df.iloc[id]['Identifier'] + '/METADATA/' + fname19115)

## Part 3. Upload RAW data to Zenodo

In [65]:
### Zip all folders
zip_each_folder(input, df)

INFO: Ziping 20230430_MDG-Nosy-Ve_UAV-01_6
    INFO: details.pix4dcapture-mission is not a folder
    INFO: GPS.zip is not a folder
    INFO: metadata_2023_04_30_Nosy_Ve_drone_julien_Mission6.csv is not a folder
    INFO: LABEL.zip is not a folder
    INFO: sync_state.json is not a folder
    INFO: 00_sample_rawdata_overview.png is not a folder
    INFO: DCIM.zip is not a folder
    INFO: METADATA.zip is not a folder
    INFO: 2023_04_30_Nosy_Ve_drone_julien_Mission6_DCMI_metadata.csv is not a folder
    INFO: PROCESSED_DATA.zip is not a folder
INFO: Ziping 20230501_MDG-Anakao-Prince-Anakao_UAV-01_4
    INFO: details.pix4dcapture-mission is not a folder
    INFO: metadata_2023_05_01_Anakao_Prince_Anakao_drone_julien_Mission4.csv is not a folder
    INFO: 2023_05_01_Anakao_Prince_Anakao_drone_julien_Mission4_DCMI_metadata.csv is not a folder
    INFO: GPS.zip is not a folder
    INFO: LABEL.zip is not a folder
    INFO: sync_state.json is not a folder
    INFO: 00_sample_rawdata_overvie

### Reference files in dataframe (data column)

In [80]:
### Reference Zip's in Dataframe
for zip in range(len(df)):
    source = []
    sess_path = os.path.join(input, df.iloc[zip]['Identifier'])
    if "doi" in sess_path:
        sess_path = os.path.join(input, sess_path.split("_\n")[0].split("id:")[1])
    source = os.listdir(sess_path)
    # print(len(source))
    # print(source)
    if len(source) < 1:
        pass
    else:
        source_file = []
        for file in range(len(source)):
            if "PROCESSED_DATA" in source[file]:
                pass
            elif "000_photogrammetry_report.pdf" in source[file]:
                pass                
            else:
                if os.path.isfile(os.path.join(sess_path, source[file])):
                    if len(source) < 2:
                        df.iloc[zip, df.columns.get_loc('Data')] = "source:" + source[file] + "@" + sess_path + "/" + source[file] + "_\n" + "sourceType:other_\nsourceZip:false"
                        source_file = source[file]
                    else:
                        source_file.append(source[file])
                        # df.iloc[zip, df.columns.get_loc('Data')] = "source:" + source[file] + "@" + sess_path + source[file] + "_\n" + "sourceType:other_\nsourceZip:false"
        if ".zip" in  df.iloc[zip, df.columns.get_loc('Data')]:
            print("zip already in Data")
            pass
        else:
            df.iloc[zip, df.columns.get_loc('Data')] = "source:"
            for i in range(len(source_file)):
                print(source_file[i], i+1, "/", len(source_file))
                df.iloc[zip, df.columns.get_loc('Data')] =  str(df.iloc[zip, df.columns.get_loc('Data')]) + source_file[i] + "@" + sess_path + "/" + source_file[i] + ","
    
    if "," in str(df.iloc[zip, df.columns.get_loc('Data')])[-1]:
        df.iloc[zip, df.columns.get_loc('Data')] = str(df.iloc[zip, df.columns.get_loc('Data')])[:-1] + "_\n" + "sourceType:other_\nupload:true"

df.iloc[zip]['Data']


details.pix4dcapture-mission 1 / 9
GPS.zip 2 / 9
metadata_2023_04_30_Nosy_Ve_drone_julien_Mission6.csv 3 / 9
LABEL.zip 4 / 9
sync_state.json 5 / 9
00_sample_rawdata_overview.png 6 / 9
DCIM.zip 7 / 9
METADATA.zip 8 / 9
2023_04_30_Nosy_Ve_drone_julien_Mission6_DCMI_metadata.csv 9 / 9
details.pix4dcapture-mission 1 / 9
metadata_2023_05_01_Anakao_Prince_Anakao_drone_julien_Mission4.csv 2 / 9
2023_05_01_Anakao_Prince_Anakao_drone_julien_Mission4_DCMI_metadata.csv 3 / 9
GPS.zip 4 / 9
LABEL.zip 5 / 9
sync_state.json 6 / 9
00_sample_rawdata_overview.png 7 / 9
DCIM.zip 8 / 9
METADATA.zip 9 / 9
Mission 14.p4d 1 / 10
2023_05_05_Madagascar_Sarodrano_Reef_Manjaka_Gauche_drone_julien_Mission14_DCMI_metadata.csv 2 / 10
details.pix4dcapture-mission 3 / 10
GPS.zip 4 / 10
LABEL.zip 5 / 10
sync_state.json 6 / 10
00_sample_rawdata_overview.png 7 / 10
DCIM.zip 8 / 10
METADATA.zip 9 / 10
metadata_2023_05_05_Madagascar_Sarodrano_Reef_Manjaka_Gauche_drone_julien_Mission14.csv 10 / 10
metadata_2023_05_01_Anaka

'source:details.pix4dcapture-mission@/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/details.pix4dcapture-mission,GPS.zip@/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/GPS.zip,2023_05_03_Nosy_Veolmitahy_drone_julien_Mission2_DCMI_metadata.csv@/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/2023_05_03_Nosy_Veolmitahy_drone_julien_Mission2_DCMI_metadata.csv,LABEL.zip@/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/LABEL.zip,sync_state.json@/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/sync_state.json,00_sample_rawdata_overview.png@/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/00_sample_rawdata_overview.png,DCIM.zip@/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/DCIM.zip,METADATA.zip@/home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2/METADATA.zip,Mission 2.p4d@/hom

### Generate config files for geoflow or re-use dataframe later

In [81]:
### Export raw data df to csv
fname = out_prefix + '_zenodo-rawdata'
df.to_csv(os.path.join(output, fname + ".csv"), quoting=1, quotechar='"', index=False)

### Configure and export Zenodo json
# SANDBOX
# zen_json = zenodo_json(fname, 'sandbox', output, out_prefix)
# PRODUCTION
zen_json = zenodo_json(fname, 'no_sandbox', output, out_prefix)
with open(os.path.join(output, out_prefix + '_rawfiles-config.json'), 'w') as f:
    json.dump(zen_json, f, indent=4)


### Execute Geoflow to upload to Zenodo
/!\ DISABLED SINCE FRIDAY 2023 10 13 /!\
/!\ Zenodo upgrade /!\

In [26]:
# ### Execute geoflow :: upload to geoflow
# # upload_raw_to_zenodo = 
# config_raw = os.path.join(output, out_prefix + '_rawfiles-config.json')
# # run_geoflow(config_raw, geoflow_folder)
# os.chdir(geoflow_folder)
# # subprocess.run(["R", "-e", "geoflow::initWorkflow('{}')".format(config_raw)], capture_output=False)
# subprocess.call(["R", "-e", "geoflow::executeWorkflow('{}')".format(config_raw)],
#                 stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT,)
# os.chdir(dronemdt_folder)

### Upload to Zenodo with Python

In [86]:
%load_ext autoreload
%autoreload 2

# del(zenul)
from zenodo_helper import *
import hashlib
# help(zenul)
# data_zip

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
### Upload to Zenodo
print("#### Upload zip files to Zenodo")
base_url = "https://zenodo.org/api/"
for zipul in range(len(df)):
    # print("Dataset:", zipul, "/" ,len(df))
    print('\nDataset:', zipul+1, "/" ,len(df), "\n    ",df.iloc[zipul]['Identifier'])
    zenodo_baseurl = base_url

    #### Extract source from Data in dataframe
    data_zip = df.iloc[zipul]['Data'].split('source:')[1].split('_\n')[0]
    if "," in data_zip:
        data_ziptemp = data_zip.split(',')
        data_zip = []
        for dt in range(len(data_ziptemp)):
            data_zip.append(data_ziptemp[dt].split('@')[0])
    else:
        data_zip = data_zip.split('@')[0]
    # data_zip = source_file
    print(data_zip)

    ### Put Metadata or verify if doi exists (Metadata not updated if doi exists)
    if 'doi:' in df.iloc[zipul]['Identifier']:
        doi_raw = df.iloc[zipul]['Identifier'].split('_\ndoi:')[1].split('\n')[0].split('.')[-1]
        print("DOI already present: ",doi_raw)
        getrecid = zenlist_single(zenodo_baseurl, ACCESS_TOKEN, str(doi_raw))
        zenval = zenvar(getrecid)
    else:
        print("Initialize deposit")
        r = check_token(zenodo_baseurl, ACCESS_TOKEN)
        zenval = zenvar(r)
        print("prereserved doi:"+zenval[1])
        print("Write DOI to dataframe")
        dfzen = df
        if 'id:' in dfzen.iloc[zipul]['Identifier']:
            pass
        else:
            dfzen.iloc[zipul, dfzen.columns.get_loc('Identifier')] = "id:" + dfzen.iloc[zipul]['Identifier'] + "_\ndoi:" + zenval[1]
            dfzen.iloc[zipul, dfzen.columns.get_loc("Provenance")] = dfzen.iloc[zipul]["Provenance"] + "_\nprocess:Raw dataset uploaded to " + base_url.split('api')[0] + "record/" + str(zenval[2])
            
        print("Enrich upload with metadata")
        zen_metadata = zenmdt(zenodo_baseurl, ACCESS_TOKEN, zenval[2], df, zipul)
        if zen_metadata.status_code > 400:
            print("error in metadata, please check there is no double keywords: \n" + zen_metadata.text)
            break
        
    # print(zen_metadata.text)       
    print("upload data")
    print("Trying upload number: 1")

    for file in data_zip:
        ul_count = 1
        ### Control Filename
        # while file not in zenlist_single_files(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2])).text:
        ### Control md5 checksum:
        ful = os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1], file)
        print(ful)
        with open(ful, "rb") as file_to_check:
            # Open,close, read file and calculate MD5 on its contents 
            # read contents of the file
            ## Python 3.10
            fh = file_to_check.read() 
            # pipe contents of the file through
            digest = hashlib.md5(fh)
            ## End Python 3.10
            # digest = hashlib.file_digest(file_to_check, "md5") ### Python 3.11 only
        print("    md5:", digest.hexdigest())
        while digest.hexdigest() not in zenlist_single_files(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2])).text:
            #Reset connection every 15 tries
            if ul_count == 15:
                getrecid = zenlist_single(zenodo_baseurl, ACCESS_TOKEN, str(doi_raw))
                zenval = zenvar(getrecid)
            
            file_on_server = zenlist_single_files(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2]))
            ### Clean wrong checksum
            if file in file_on_server.text:
                print("    clean wrong files")
                for fs in range(len(file_on_server.json())):
                    if file == file_on_server.json()[fs]['filename']:
                        # print("    sleep 5: computing checksum on server")
                        # time.sleep(15)
                        print("    checksum on server", file_on_server.json()[fs]['checksum'])
                        furl = file_on_server.json()[fs]["links"]["self"]
                        file_to_remove = zen_del_file(furl, ACCESS_TOKEN)
                        
            file_list = [file]            
            zen_upload = zenul(zenval[0], ACCESS_TOKEN, os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1]), file_list)
            print("        ", zen_upload.text)
            if zen_upload.status_code == 404:
                print("Version doesn't exists ! Please check your record_id")
                break
            else:
                print("    Retry number: " + str(ul_count))                
                ul_count += 1
                if zen_upload.status_code == 403:
                    print("        Friday 13th 2023 nightmare => permission denied!")
                    time.sleep(5)      
            
    # zen_upload = zenul(zenval[0], ACCESS_TOKEN, os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1]), data_zip)
    # # for file in data_zip:
    # #     print("Uploading: ", file)
    # #     print("curl", "--upload-file", os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1], file), zenval[0] + "/" + file + "?" + "k3LQP8uj4HtpRbjMMXMIUBeRJr0UFELUwjyXWIakTHh0xP9p5PhZ7bzCqRN1")
    # #     subprocess.check_output(["curl", "--upload-file", 
    # #                              os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1], file),
    # #                              zenval[0] + "/" + file + "?" + "k3LQP8uj4HtpRbjMMXMIUBeRJr0UFELUwjyXWIakTHh0xP9p5PhZ7bzCqRN1"
    # #                                           ])
    # if zen_upload.status_code != 201:
    #     for file in data_zip:
    #         file_list = [file]
    #         zen_upload = zenul(zenval[0], ACCESS_TOKEN, os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1]), file_list)
    #         # zen_upload = zenul(zenvar(newv), ACCESS_TOKEN, os.getcwd(), test_list)
    #         # zen_upload = zenul2(zenodo_baseurl, str(zenval[-1]), ACCESS_TOKEN, os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1]), data_zip)
    #         print("Upload status OK? " + str(zen_upload.ok))
    #         print("Control files: first pass")
    #         ul_count = 1
    #         while zen_upload.status_code != 201:
    #             if zen_upload.status_code == 404:
    #                 print("Version doesn't exists ! Please check your record_id")
    #                 break
    #             else:
    #                 ul_count += 1
    #                 print("Retry number: " + str(ul_count))
    #                 zen_upload = zenul(zenval[0], ACCESS_TOKEN, os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1]), file_list)
    #                 if zen_upload.status_code == 403:
    #                     print("        Friday 13th 2023 nightmare => permission denied!")
    #                     time.sleep(5)
                    
            # zen_metadata = zenmdt(zenodo_baseurl, ACCESS_TOKEN, zenval[2], df, zipul)
            ## Too agressive                
            # print("Control files: second pass")
            # if zen_upload.status_code == 201:
            #     # files_ul = zenfiles(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2]))
            #     # while len(zenfiles(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2])).json()) != len(data_zip):
            #     while file not in zenfiles(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2])).json():
            #         if zen_upload.status_code == 404:
            #             print("Version doesn't exists ! Please check your record_id")
            #             break
            #         else:
            #             ul_count += 1
            #             print("Retry number: " + str(ul_count))
            #             zen_upload = zenul(zenval[0], ACCESS_TOKEN, os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1]), file_list)
            #             # files_ul = zenfiles(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2]))       
            

    # ### Publish to Zenodo
    # print("Publish record")
    # zenpublish(zenodo_baseurl, ACCESS_TOKEN, zenval[2])

#### Upload zip files to Zenodo

Dataset: 1 / 34 
     id:20230430_MDG-Nosy-Ve_UAV-01_6_
doi:10.5281/zenodo.10182430
['details.pix4dcapture-mission', 'GPS.zip', 'metadata_2023_04_30_Nosy_Ve_drone_julien_Mission6.csv', 'LABEL.zip', 'sync_state.json', '00_sample_rawdata_overview.png', 'DCIM.zip', 'METADATA.zip', '2023_04_30_Nosy_Ve_drone_julien_Mission6_DCMI_metadata.csv']
DOI already present:  10182430
upload data
Trying upload number: 1
/home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-01_6/details.pix4dcapture-mission
    md5: 9bc82a098ffcaa4d59066b206e2984ea
    Sleep 5 seconds before new upload
upload: details.pix4dcapture-mission
         {"created": "2023-11-21T18:33:02.025264+00:00", "updated": "2023-11-21T18:33:02.121487+00:00", "version_id": "a8e85ab7-e3b1-461e-aca5-7a12b550ea0c", "key": "details.pix4dcapture-mission", "size": 7969, "mimetype": "application/octet-stream", "checksum": "md5:9bc82a098ffcaa4d59066b206e2984ea", "is_head": true, "delete_marker": false, "

In [103]:
### Control on last upload
print("Uploaded files available for record:", zenval[2])
a = zenlist_single_files(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2]))
for i in range(len(a.json())):
    print("  |__" ,a.json()[i]['filename'])

Uploaded files available for record: 10182496
  |__ DCIM.zip
  |__ sync_state.json
  |__ GPS.zip
  |__ details.pix4dcapture-mission
  |__ 00_sample_rawdata_overview.png
  |__ METADATA.zip
  |__ 2023_05_03_Nosy_Veolmitahy_drone_julien_Mission2_DCMI_metadata.csv
  |__ LABEL.zip
  |__ Mission 2.p4d


In [104]:
### Check DOI's

for doi in range(len(df)):
    print(df.iloc[doi]["Identifier"])

id:20230430_MDG-Nosy-Ve_UAV-01_6_
doi:10.5281/zenodo.10182430
id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_4_
doi:10.5281/zenodo.10182432
id:20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_14_
doi:10.5281/zenodo.10182434
id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_7_
doi:10.5281/zenodo.10182436
id:20230507_MDG-Mangily-estran_UAV-01_3_
doi:10.5281/zenodo.10182438
id:20230502_MDG-Sarodrano-IOT_UAV-01_5_
doi:10.5281/zenodo.10182440
id:20230430_MDG-Nosy-Ve_UAV-01_5_
doi:10.5281/zenodo.10182442
id:20230505_MDG-Sarodrano-village_UAV-01_1_
doi:10.5281/zenodo.10182444
id:20230504_MDG-GRT-micro-atoll-zone-B_UAV-01_1_
doi:10.5281/zenodo.10182446
id:20230503_MDG-Sarodrano-IOT_UAV-01_11_
doi:10.5281/zenodo.10182448
id:20230502_MDG-Sarodrano-IOT_UAV-01_6_
doi:10.5281/zenodo.10182450
id:20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_15_
doi:10.5281/zenodo.10182452
id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_6_
doi:10.5281/zenodo.10182454
id:20230430_MDG-Nosy-Ve_UAV-01_3_
doi:10.5281/zenodo.10182

In [105]:
### Export Dataframe to csv
# ### Export Dataframe to csv
csv_published = out_prefix + "_zenodo-rawdata"
df.to_csv(os.path.join(output, csv_published + ".csv"), quoting=1, quotechar='"', index=False)
# ### Export geoflow config to json
if sandbox == True:
    zenodo_url = "https://sandbox.zenodo.org/api"
else:
    zenodo_url = "https://zenodo.org/api"
rawdata_config = {'profile': {'id': 'rawdata', 'name': 'rawdata', 'project': 'Publish rawdata with Geoflow', 'organization': 'IRD', 'environment': {'file': '.env', 'hide_env_vars': ['MOTDEPASSE']}, 'logos': ['https://en.ird.fr/sites/ird_fr/files/2019-08/logo_IRD_2016_BLOC_UK_COUL.png'], 'mode': 'entity'}, 'metadata': {'entities': [{'handler': 'csv', 'source': os.path.join(output, out_prefix) + "_zenodo-rawdata.csv"}], 'contacts': [{'handler': 'csv', 'source': 'https://drive.ird.fr/s/EYS3qccyB28PrA9/download/geoflow_g2oi_contacts.csv'}]}, 'software': [{"id": "my-zenodo","type": "output","software_type": "zenodo","parameters": {"url": zenodo_url,"token": "{{ ZENODO_TOKEN }}","logger": "DEBUG"},"properties": {"clean": {"run": False}}}], 'actions': [{"id": "zen4R-deposit-record","options": {"update_files": True,"communities": "uav","depositWithFiles": True,"publish": False,"update_metadata": True,"strategy": "newversion","deleteOldFiles": True},"run": True}]}
with open(os.path.join(output, out_prefix + '_zenodo-rawdata-config.json'), 'w') as f:
    json.dump(rawdata_config, f, indent=4)

## Part 4. Download from ODM

In [106]:
# Take csv generated from python's zenodo upload as updated dataframe
df = {}
df = pd.read_csv(os.path.join(input, out_prefix + '_zenodo-rawdata.csv'))
df.head()

,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,id:20230430_MDG-Nosy-Ve_UAV-01_6_\ndoi:10.5281/zenodo.10182430,"title:Images drone UAV, Nosy ve, Madagascar - 20230430 - 01_6","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Nosy v...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-04-30 14-06-14 - 2023-04-30 14-11-01,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:details.pix4dcapture-mission@/home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-01...
1,id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_4_\ndoi:10.5281/zenodo.10182432,"title:Images drone UAV, Anakao prince anakao, Madagascar - 20230501 - 01_4","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Anakao...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-01 08-29-43 - 2023-05-01 08-43-12,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:details.pix4dcapture-mission@/home/sylvain/HDD5To/DATA/Mada/julien/20230501_MDG-Anakao-Prince-...
2,id:20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_14_\ndoi:10.5281/zenodo.10182434,"title:Images drone UAV, Sarodrano reef manjaka gauche, Madagascar - 20230505 - 01_14","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Sarodr...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-05 08-56-23 - 2023-05-05 09-01-04,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:Mission 14.p4d@/home/sylvain/HDD5To/DATA/Mada/julien/20230505_MDG-Sarodrano-Reef-Manjaka-Gauch...
3,id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_7_\ndoi:10.5281/zenodo.10182436,"title:Images drone UAV, Anakao prince anakao, Madagascar - 20230501 - 01_7","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Anakao...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-01 17-51-24 - 2023-05-01 17-56-17,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ir

In [107]:
### SEARCH AND DOWNLOAD FROM ODM
odm_projects = list_all_projects()
for projects in range(len(odm_projects)):
    for p in range(len(df)):
        if df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1] in odm_projects[projects]['name']:
            print(projects, odm_projects[projects]['id'], odm_projects[projects]['name'], odm_projects[projects]['tasks'])
            project_id = odm_projects[projects]['id']
            tasks =  odm_projects[projects]['tasks']
            res = requests.post(settings.SERVER + '/api/token-auth/', 
                            data={'username': settings.USERNAME,
                                'password': settings.PASSWORD}).json()
            if 'token' in res:
                print("Still logged-in!")
                token = res['token']
                for task in range(len(tasks)):
                    task_id = tasks[task]
                    print(task_id)
                    res = requests.get(settings.SERVER + "/api/projects/{}/tasks/{}/download/all.zip".format(project_id, task_id), 
                                    headers={'Authorization': 'JWT {}'.format(token)},
                                    stream=True)
                    # token = res['token']
                    sess_path = os.path.join(input, df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1])
                    file_newname = 'PROCESSED_DATA.zip'
                    ### Reading PROCESSED_DATA folder
                    ### Check and create PROCESSED_DATA Directory if it not exists
                    # process_dir=os.path.join(sess_path, 'PROCESSED_DATA')
                    # if os.path.exists(process_dir):
                    if os.path.exists(sess_path):
                        if 'PROCESSED_DATA.zip' in os.listdir(sess_path):
                            print("INFO: data already found in {} !".format(df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1]))
                        else:
                        #     print("INFO: Create PROCESSED_DATA folder in {} folder")
                        #     os.mkdir(process_dir)
                            print("Downloading all.zip")
                            with open(os.path.join(input, df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1], "all.zip"), 'wb') as f:
                                for chunk in res.iter_content(chunk_size=1024): 
                                    if chunk:
                                        f.write(chunk)
                            # file_newname = "ortho_" + df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1] + ".tif"
                            # file_newname = 'PROCESSED_DATA.zip'
                            print("Renaming all.zip to PROCESSED_DATA.zip")
                            os.rename(os.path.join(sess_path, "all.zip"), os.path.join(sess_path, file_newname))
                            print("Saved " + file_newname)

                        res = requests.get(settings.SERVER + "/api/projects/{}/tasks/{}/download/report.pdf".format(project_id, task_id), 
                                            headers={'Authorization': 'JWT {}'.format(token)},
                                            stream=True)
                        pdf_newname = '000_photogrammetry_report.pdf'
                        if pdf_newname in os.listdir(sess_path):
                            print("INFO: PDF already found in {} !".format(df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1]))
                        else:
                            print("Downloading report.pdf")
                            with open(os.path.join(input, df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1], "report.pdf"), 'wb') as pdf:
                                for chunk in res.iter_content(chunk_size=1024): 
                                    if chunk:
                                        pdf.write(chunk)
                            
                            os.rename(os.path.join(sess_path, "report.pdf"), os.path.join(sess_path, pdf_newname))
                            print("Saved " + pdf_newname)                    
            ### Enrich Metadata
            dfdlodm = df
            dfdlodm.iloc[p, dfdlodm.columns.get_loc('Provenance')] = dfdlodm.iloc[p]["Provenance"] + "_\nprocess:Download from WebODM"
            dfdlodm.iloc[p, dfdlodm.columns.get_loc('Format')] = "resource:image/tiff_\ndistribution:image/tiff[GeoTIFF]"
            dfdlodm.iloc[p, dfdlodm.columns.get_loc('Data')] = pdf_newname + "@"+ os.path.join(sess_path, pdf_newname) + "," + file_newname + "@"+ os.path.join(sess_path, file_newname) + "_\nsourceType:other_\nsourceZip:False"

Successfully Logged-in!
########
id name permissions
----------
227 20220911_MYT-jollyroger_UAV-02_1
224 20220911_MYT-musicalplage_UAV-02_3
223 20220911_MYT-jollyroger_UAV-02_2
222 20220912_MYT-ile-blanche_UAV-02_1
221 20220913_MYT-ngouja_UAV-02_3
219 20220913_MYT-jollyroger_UAV-02_1
217 20220910_MYT-jollyroger_UAV-02_2
215 20220910_MYT-jolly-roger_UAV-02_3
214 20220914_MYT-ile-blanche_UAV-02_2
213 20220912_MYT-surprise_UAV-02_3
212 20220911_MYT-aeroport_UAV-02_1
209 20220912_MYT-surprise_UAV-02_2
208 20220910_MYT-musicalplage_UAV-02_1
207 20220911_MYT-aeroport_UAV-02_2
199 ALD_FRAMES_SUBSET_-9.3734_46.2131_2transectsFilt_RAW
198 ALD_FRAMES_SUBSET_-9.3734_46.2131_2transectsFilt
197 ALD_FRAMES_SUBSET_-9.3734_46.2131_rad_30_1698667597s
196 PyWebODM - 20230430_MDG-Nosy-Ve_UAV-01_3
195 PyWebODM - 20230502_MDG-Sarodrano-IOT_UAV-01_5
194 PyWebODM - 20230427_MDG-Toliara-Mangroves_UAV-01_1
193 20230430_MDG-Nosy-Ve_UAV-01_6
189 PyWebODM - 20230916_MDG-nosy-sakatia_UAV-02_1
188 20230906_REU-BOUC

In [108]:
### Reformat dataframe

for entities in range(len(df)):
    sess_path = os.path.join(
        input, df.iloc[entities]["Identifier"].split("_\n")[0].split(":")[1]
    )
    df.iloc[entities, df.columns.get_loc("Title")] = df.iloc[entities][
        "Title"
    ].replace(
        "Images drone UAV,",
        "Orthophoto & DEM (MNE) issues d'images drone, UAV,",
    )
    df.iloc[entities, df.columns.get_loc("Description")] = df.iloc[entities][
        "Description"
    ].replace(
        "Jeu de données composé d’images",
        "Ce jeu de données présente les résultats des traitements photogrammétriques d’images",
    )
    df.iloc[entities, df.columns.get_loc("Description")] = df.iloc[entities][
        "Description"
    ].replace(
        "Les vols ont été réalisés dans le but",
        "Les traitements ont été réalisés avec le logiciel OpenDroneMap à partir des images brutes fournies dans la première version de ce DOI.\n<br />Les vols ont été réalisés dans le but",
    )
    df.iloc[entities, df.columns.get_loc("Description")] = df.iloc[entities][
        "Description"
    ].replace(
        "<br /><b>Le dépôt est composé",
        "<br /><br />Le paramétrage du logiciel OpenDroneMap est partagé pour permettre la reproductibilité ou l'amélioration des traitements proposés:<br />\n[\n<br />        {\n<br />            'name': 'orthophoto-resolution',\n<br />            'value': 1\n<br />        },\n<br />        {\n<br />            'name': 'auto-boundary',\n<br />            'value': true\n<br />        },\n<br />        {\n<br />            'name': 'dem-resolution',\n<br />            'value': '2.0'\n<br />        },\n<br />        {\n<br />            'name': 'dsm',\n<br />            'value': true\n<br />        }\n<br />    ]\n<br />\n<br /><b>Le dépôt est composé",
    )
    df.iloc[entities, df.columns.get_loc("Description")] = df.iloc[entities][
        "Description"
    ].replace(
        "<br />    - METADATA.zip: Métadonnées au format ISO19115, Rapports avec miniatures des images de drone (dossier tb) et statistiques de vols.",
        "<br />    - METADATA.zip: Métadonnées au format ISO19115, Rapports avec miniatures des images de drone (dossier tb) et statistiques de vols.\n<br />    - PROCESSED_DATA.zip: Orthophoto, DEM, nuages de points, ...",
    )
    df.iloc[entities, df.columns.get_loc("Description")] = df.iloc[entities][
        "Description"
    ].replace(
        "<br />│----------------  └─ tb",
        "<br />│----------------  └─ tb\n<br />│--------  └─ PROCESSED_DATA",
    )
    df.iloc[entities, df.columns.get_loc("Description")] = df.iloc[entities][
        "Description"
    ].replace(
        "\n<br />Les dépôts suivants comprennent les données traitées avec OpenDroneMap",
        "",
    )
    df.iloc[entities, df.columns.get_loc("Description")] = df.iloc[entities][
        "Description"
    ].replace(
        "\n<br />D'autres versions plus récentes de ce dépôt sont disponibles. Ces dernières comprennent notamment les données traitées avec OpenDroneMap",
        "",
    )
    df.iloc[entities, df.columns.get_loc("Description")] = df.iloc[entities][
        "Description"
    ].replace("RAW DATA", "PROCESSED DATA")
    df.iloc[entities, df.columns.get_loc("Provenance")] = (
        df.iloc[entities]["Provenance"] + "_\nprocess:Download from WebODM"
    )
    df.iloc[
        entities, df.columns.get_loc("Format")
    ] = "resource:image/tiff_\ndistribution:image/tiff[GeoTIFF]"
    df.iloc[entities, df.columns.get_loc("Data")] = (
        "source:"
        + pdf_newname
        + "@"
        + os.path.join(sess_path, pdf_newname)
        + ","
        + file_newname
        + "@"
        + os.path.join(sess_path, file_newname)
        + "_\nsourceType:other"
    )


In [109]:
print(df.iloc[0]["Identifier"])
print(df.iloc[0]["Title"])
print(df.iloc[0]["Description"])
print(df.iloc[0]["Data"])

id:20230430_MDG-Nosy-Ve_UAV-01_6_
doi:10.5281/zenodo.10182430
title:Orthophoto & DEM (MNE) issues d'images drone, UAV, Nosy ve, Madagascar - 20230430 - 01_6
abstract:"Ce jeu de données présente les résultats des traitements photogrammétriques d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Nosy ve, Madagascar à la date suivante : 20230430.
<br />Les vols ont été réalisés en partenariat avec l'IH.SM dans le but de créer des modèles numériques d'élévations pour cartographier l'écosystème marin.
<br />
<br /><br />Le paramétrage du logiciel OpenDroneMap est partagé pour permettre la reproductibilité ou l'amélioration des traitements proposés:<br />
[
<br />        {
<br />            'name': 'orthophoto-resolution',
<br />            'value': 1
<br />        },
<br />        {
<br />            'name': 'auto-boundary',
<br />            'value': true
<br />        },
<br />        {
<br />            'name': 'dem-resolution',
<br />            'value': '2.0'
<br />        },

In [110]:
### Export processed data df to csv
# fname = out_prefix + '_zenodo-rawdata'
# df.to_csv(os.path.join(output, fname + ".csv"), quoting=1, quotechar='"', index=False)
df.to_csv(os.path.join(output, out_prefix + "_processed_data.csv"), quoting=1, quotechar='"', index=False)

In [111]:
# Reload df from WebODM download as updated dataframe
df = {}
df = pd.read_csv(os.path.join(input, out_prefix + '_processed_data.csv'))
df.head()

,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,id:20230430_MDG-Nosy-Ve_UAV-01_6_\ndoi:10.5281/zenodo.10182430,"title:Orthophoto & DEM (MNE) issues d'images drone, UAV, Nosy ve, Madagascar - 20230430 - 01_6","abstract:""Ce jeu de données présente les résultats des traitements photogrammétriques d’images de dro...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-04-30 14-06-14 - 2023-04-30 14-11-01,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/tiff_\ndistribution:image/tiff[GeoTIFF],source:000_photogrammetry_report.pdf@/home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-0...
1,id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_4_\ndoi:10.5281/zenodo.10182432,"title:Orthophoto & DEM (MNE) issues d'images drone, UAV, Anakao prince anakao, Madagascar - 20230501 ...","abstract:""Ce jeu de données présente les résultats des traitements photogrammétriques d’images de dro...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-01 08-29-43 - 2023-05-01 08-43-12,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/tiff_\ndistribution:image/tiff[GeoTIFF],source:000_photogrammetry_report.pdf@/home/sylvain/HDD5To/DATA/Mada/julien/20230501_MDG-Anakao-Prince...
2,id:20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_14_\ndoi:10.5281/zenodo.10182434,"title:Orthophoto & DEM (MNE) issues d'images drone, UAV, Sarodrano reef manjaka gauche, Madagascar - ...","abstract:""Ce jeu de données présente les résultats des traitements photogrammétriques d’images de dro...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-05 08-56-23 - 2023-05-05 09-01-04,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/tiff_\ndistribution:image/tiff[GeoTIFF],source:000_photogrammetry_report.pdf@/home/sylvain/HDD5To/DATA/Mada/julien/20230505_MDG-Sarodrano-Ree...
3,id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_7_\ndoi:10.5281/zenodo.10182436,"title:Orthophoto & DEM (MNE) issues d'images drone, UAV, Anakao prince anakao, Madagascar - 20230501 ...","abstract:""Ce jeu de données présente les résultats des traitements photogrammétriques d’images de dro...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-01 17-51-24 - 2023-05-01 17-56-17,thumbn

In [112]:
data_zip = []
sources = df.iloc[0]['Data'].split('_\n')[0].split(':')[1].split(',')
for data_file in sources:
    data_zip.append(data_file.split('@')[0])

print(data_zip)

['000_photogrammetry_report.pdf', 'PROCESSED_DATA.zip']


In [113]:
### UNZIP PROCESSED_DATA
from zip_helper import *
for entities in range(len(df)):
    print("unzip PROCESSED_DATA.zip in: ", sess_path)
    sess_path = os.path.join(
        input, df.iloc[entities]["Identifier"].split("_\n")[0].split(":")[1]
    )
    try:
        unzip(sess_path, 'PROCESSED_DATA.zip', 'PROCESSED_DATA/PHOTOGRAMMETRY/')
    except:
        print("no_zip in: ", sess_path)
    print("Remove zip")
    try:
        os.remove(os.path.join(sess_path, 'PROCESSED_DATA.zip'))
    except:
        print("no_zip to remove in: ", sess_path)
        
    print("Recreate zip")
    zip_folder(os.path.join(sess_path, 'PROCESSED_DATA'), os.path.join(sess_path, 'PROCESSED_DATA'))
    

unzip PROCESSED_DATA.zip in:  /home/sylvain/HDD5To/DATA/Mada/julien/20230503_MDG-Nosy-Veolmitahy_UAV-01_2
Remove zip
Recreate zip
unzip PROCESSED_DATA.zip in:  /home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-01_6
Remove zip
Recreate zip
unzip PROCESSED_DATA.zip in:  /home/sylvain/HDD5To/DATA/Mada/julien/20230501_MDG-Anakao-Prince-Anakao_UAV-01_4
Remove zip
Recreate zip
unzip PROCESSED_DATA.zip in:  /home/sylvain/HDD5To/DATA/Mada/julien/20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_14
Remove zip
Recreate zip
unzip PROCESSED_DATA.zip in:  /home/sylvain/HDD5To/DATA/Mada/julien/20230501_MDG-Anakao-Prince-Anakao_UAV-01_7
Remove zip
Recreate zip
unzip PROCESSED_DATA.zip in:  /home/sylvain/HDD5To/DATA/Mada/julien/20230507_MDG-Mangily-estran_UAV-01_3
Remove zip
Recreate zip
unzip PROCESSED_DATA.zip in:  /home/sylvain/HDD5To/DATA/Mada/julien/20230502_MDG-Sarodrano-IOT_UAV-01_5
Remove zip
Recreate zip
unzip PROCESSED_DATA.zip in:  /home/sylvain/HDD5To/DATA/Mada/julien/20230430

In [178]:
df.iloc[0]['Identifier'].split('\n')[1].split('.')[-1]

'10037633'

### Geoflow to upload processed data

In [143]:
# ### Execute geoflow :: upload to geoflow
# # config_raw = os.path.join(output, out_prefix + '_rawfiles-config.json')
# pdata = os.path.join(output, 'comores_processed.json')
# # run_geoflow(config_raw, geoflow_folder)
# os.chdir(geoflow_folder)
# # subprocess.run(["R", "-e", "geoflow::initWorkflow('{}')".format(config_raw)], capture_output=False)
# subprocess.call(["R", "-e", "geoflow::executeWorkflow('{}')".format(pdata)],
#                 stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT,)
# os.chdir(dronemdt_folder)

### Upload processed data with python

In [198]:
### Upload processed_data to Zenodo
print("#### Upload zip files to Zenodo")
base_url = "https://zenodo.org/api/"
for zipul in range(len(df)):
    print('\nDataset:', zipul+1, "/" ,len(df), "\n    ",df.iloc[zipul]['Identifier'])
    zenodo_baseurl = base_url
    
    #### Extract source from Data in dataframe
    data_zip = df.iloc[zipul]['Data'].split('source:')[1].split('_\n')[0]
    if "," in data_zip:
        data_ziptemp = data_zip.split(',')
        data_zip = []
        for dt in range(len(data_ziptemp)):
            data_zip.append(data_ziptemp[dt].split('@')[0])
    else:
        data_zip = data_zip.split('@')[0]
    # data_zip = source_file
    print(data_zip)
    
    
    if 'doi:' in df.iloc[zipul]['Identifier']:
        doi_raw = df.iloc[zipul]['Identifier'].split('_\ndoi:')[1].split('\n')[0].split('.')[-1]
        print("DOI already present: ",doi_raw)
        getrecid = zenlist_single(zenodo_baseurl, ACCESS_TOKEN, str(doi_raw))
        
        if int(doi_raw) == int(getrecid.json()['conceptdoi'].split('.')[-1]) + 1 :
            print("New deposit version")
            newv = zen_newversion(zenodo_baseurl, ACCESS_TOKEN, str(doi_raw))
            zenval = zenvar(newv)
            print("New doi version:"+zenval[1])

            print("Write DOI to dataframe")
            df.iloc[zipul, df.columns.get_loc('Identifier')] = "id:" + df.iloc[zipul]['Identifier'].split('_\n')[0].split('id:')[1] + "_\ndoi:" + zenval[1]
            
            print("Enrich upload with metadata")
            df.iloc[zipul, df.columns.get_loc("Provenance")] = df.iloc[zipul]["Provenance"] + "_\nprocess:Processed data uploaded to " + base_url.split('api')[0] + "record/" + str(zenval[2])
        
        else:
            print('Version in draft status')
            zenval = zenvar(getrecid)
        
        print("Enrich upload with metadata")
        zen_metadata = zenmdt(zenodo_baseurl, ACCESS_TOKEN, zenval[2], df, zipul)
        
    else:
        print("Initialize deposit")
        r = check_token(zenodo_baseurl, ACCESS_TOKEN)
        zenval = zenvar(r)
        print("prereserved doi:"+zenval[1])
        print("Write DOI to dataframe")
        dfzen = df
        if 'doi:' in dfzen.iloc[zipul]['Identifier']:
            pass
        else:
            dfzen.iloc[zipul, dfzen.columns.get_loc('Identifier')] = "id:" + dfzen.iloc[zipul]['Identifier'] + "_\ndoi:" + zenval[1]
            dfzen.iloc[zipul, dfzen.columns.get_loc("Provenance")] = dfzen.iloc[zipul]["Provenance"] + "_\nprocess:Processed dataset uploaded to " + base_url.split('api')[0] + "record/" + str(zenval[2])
            
        print("Enrich upload with metadata")
        zen_metadata = zenmdt(zenodo_baseurl, ACCESS_TOKEN, zenval[2], df, zipul)
        if zen_metadata.status_code > 400:
            print("error in metadata, please check there is no double keywords: \n" + zen_metadata.text)
            break 

    print("upload data")
    print("    Trying upload number: 1")
    
    for file in data_zip:
        ul_count = 1
        ### Control Filename
        # while file not in zenlist_single_files(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2])).text:
        ### Control md5 checksum:
        ful = os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1], file)
        print(ful)
        with open(ful, "rb") as file_to_check:
            # Open,close, read file and calculate MD5 on its contents 
            # read contents of the file
            ## Python 3.10
            fh = file_to_check.read() 
            # pipe contents of the file through
            digest = hashlib.md5(fh)
            ## End Python 3.10
            # digest = hashlib.file_digest(file_to_check, "md5") ### Python 3.11 only
        print("    md5:", digest.hexdigest())
        while digest.hexdigest() not in zenlist_single_files(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2])).text:
            #Reset connection every 15 tries
            if ul_count == 15:
                getrecid = zenlist_single(zenodo_baseurl, ACCESS_TOKEN, str(doi_raw))
                zenval = zenvar(getrecid)
            
            file_on_server = zenlist_single_files(zenodo_baseurl, ACCESS_TOKEN, str(zenval[2]))
            ### Clean wrong checksum
            if file in file_on_server.text:
                print("    clean wrong files")
                for fs in range(len(file_on_server.json())):
                    if file == file_on_server.json()[fs]['filename']:
                        # print("    sleep 5: computing checksum on server")
                        # time.sleep(15)
                        print("    checksum on server", file_on_server.json()[fs]['checksum'])
                        furl = file_on_server.json()[fs]["links"]["self"]
                        file_to_remove = zen_del_file(furl, ACCESS_TOKEN)
                        
            file_list = [file]            
            zen_upload = zenul(zenval[0], ACCESS_TOKEN, os.path.join(input, df.iloc[zipul]['Identifier'].split('_\n')[0].split(':')[1]), file_list)
            print("        ", zen_upload.text)
            if zen_upload.status_code == 404:
                print("Version doesn't exists ! Please check your record_id")
                break
            else:
                ul_count += 1
                print("    Retry number: " + str(ul_count))
                if zen_upload.status_code == 403:
                    print("        Friday 13th 2023 nightmare => permission denied!")
                    time.sleep(5)    

    ### Publish to Zenodo
    print("Publish record")
    published = zenpublish(zenodo_baseurl, ACCESS_TOKEN, zenval[2])
    print(published.text)

#### Upload zip files to Zenodo
['000_photogrammetry_report.pdf', 'PROCESSED_DATA.zip']
New deposit version
New doi version:10.5281/zenodo.10039105
Write DOI to dataframe
Enrich upload with metadata
upload data
    Trying upload number: 1
#### file {} not in repository.
#### upload!
Sleep 1 second before new upload
upload: 000_photogrammetry_report.pdf
        Upload status OK? True
        Control files: first pass
#### file {} not in repository.
#### upload!
Sleep 1 second before new upload
upload: PROCESSED_DATA.zip
        Upload status OK? True
        Control files: first pass
['000_photogrammetry_report.pdf', 'PROCESSED_DATA.zip']
New deposit version
New doi version:10.5281/zenodo.10039106
Write DOI to dataframe
Enrich upload with metadata
upload data
    Trying upload number: 1
#### file {} not in repository.
#### upload!
Sleep 1 second before new upload
upload: 000_photogrammetry_report.pdf
        Upload status OK? True
        Control files: first pass
#### file {} not in re

In [ ]:
### Check DOI's

for doi in range(len(df)):
    print(df.iloc[doi]["Identifier"])

In [199]:
### Export processed data df to csv
# fname = out_prefix + '_zenodo-rawdata'
# df.to_csv(os.path.join(output, fname + ".csv"), quoting=1, quotechar='"', index=False)
df.to_csv(os.path.join(output, out_prefix + "_processed_data_updated_doi.csv"), quoting=1, quotechar='"', index=False)
# # config_raw = os.path.join(output, out_prefix + '_rawfiles-config.json')


### Update RAW Metadata 

In [115]:
# Reload Raw Dataframe
ouprefix = '20231101-161132'
df = {}
df = pd.read_csv(os.path.join(input, out_prefix + '_zenodo-rawdata.csv'))
df.head()

,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,id:20230430_MDG-Nosy-Ve_UAV-01_6_\ndoi:10.5281/zenodo.10182430,"title:Images drone UAV, Nosy ve, Madagascar - 20230430 - 01_6","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Nosy v...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-04-30 14-06-14 - 2023-04-30 14-11-01,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:details.pix4dcapture-mission@/home/sylvain/HDD5To/DATA/Mada/julien/20230430_MDG-Nosy-Ve_UAV-01...
1,id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_4_\ndoi:10.5281/zenodo.10182432,"title:Images drone UAV, Anakao prince anakao, Madagascar - 20230501 - 01_4","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Anakao...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-01 08-29-43 - 2023-05-01 08-43-12,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:details.pix4dcapture-mission@/home/sylvain/HDD5To/DATA/Mada/julien/20230501_MDG-Anakao-Prince-...
2,id:20230505_MDG-Sarodrano-Reef-Manjaka-Gauche_UAV-01_14_\ndoi:10.5281/zenodo.10182434,"title:Images drone UAV, Sarodrano reef manjaka gauche, Madagascar - 20230505 - 01_14","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Sarodr...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-05 08-56-23 - 2023-05-05 09-01-04,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:Mission 14.p4d@/home/sylvain/HDD5To/DATA/Mada/julien/20230505_MDG-Sarodrano-Reef-Manjaka-Gauch...
3,id:20230501_MDG-Anakao-Prince-Anakao_UAV-01_7_\ndoi:10.5281/zenodo.10182436,"title:Images drone UAV, Anakao prince anakao, Madagascar - 20230501 - 01_7","abstract:""Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Anakao...","theme[General]:Madagascar,drone,lagon,lagoon,corail,reef,G2OI,grand observatoire de l'océan Indien,ru...","author:julien.barde@ird.fr,sylvain.poulain@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain....",publication:2023-11-21_\nedition:2023-11-21,dataset,fra,NaN,2023-05-01 17-51-24 - 2023-05-01 17-56-17,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ir

In [116]:
print("##### BEFORE #####\n" + df.iloc[0]["Creator"])
for row in range(len(df)):
    df.iloc[row, df.columns.get_loc("Creator")] = 'author:sylvain.poulain@ird.fr,ainaledon294@gmail.com,a.herinomena11@gmail.com,md.nodc@ihsm.mg,julien.barde@ird.fr_\npublisher:sylvain.poulain@ird.fr_\nowner:sylvain.poulain@ird.fr_\npointOfContact:julien.barde@ird.fr_\nprocessor:sylvain.poulain@ird.fr'

print("##### AFTER #####\n" + df.iloc[0]["Creator"])

##### BEFORE #####
author:julien.barde@ird.fr,sylvain.poulain@ird.fr_
publisher:sylvain.poulain@ird.fr_
owner:sylvain.poulain@ird.fr_
pointOfContact:julien.barde@ird.fr_
processor:sylvain.poulain@ird.fr
##### AFTER #####
author:sylvain.poulain@ird.fr,ainaledon294@gmail.com,a.herinomena11@gmail.com,md.nodc@ihsm.mg,julien.barde@ird.fr_
publisher:sylvain.poulain@ird.fr_
owner:sylvain.poulain@ird.fr_
pointOfContact:julien.barde@ird.fr_
processor:sylvain.poulain@ird.fr


In [119]:
### Update Metadata
print("#### Update metadata")
base_url = "https://zenodo.org/api/"
for zipul in range(len(df)):
    print('\nDataset:', zipul+1, "/" ,len(df), "\n    ",df.iloc[zipul]['Identifier'])
    zenodo_baseurl = base_url 
    
    if 'doi:' in df.iloc[zipul]['Identifier']:
        doi_raw = df.iloc[zipul]['Identifier'].split('_\ndoi:')[1].split('_\n')[0].split('.')[-1]
        # print("\nDOI already present: ",doi_raw)
        # getrecid = zenlist_single(zenodo_baseurl, ACCESS_TOKEN, str(doi_raw))
        # print(getrecid.text)
        
        # zenval = zenvar(getrecid)
        # print(zenval)
        # print("\nEnrich upload with metadata")
        zenedit(zenodo_baseurl, ACCESS_TOKEN, doi_raw)
        zen_metadata = zenmdt(zenodo_baseurl, ACCESS_TOKEN, doi_raw, df, zipul, contact_file="/home/sylvain/DroneMDT/geoflow_g2oi_contacts.csv")
        print(zen_metadata)

    ### Publish to Zenodo
    # print("Publish record")
    # published = zenpublish(zenodo_baseurl, ACCESS_TOKEN, doi_raw)
    # print(published.text)

#### Update metadata

Dataset: 1 / 34 
     id:20230430_MDG-Nosy-Ve_UAV-01_6_
doi:10.5281/zenodo.10182430
Using contact file
  Looking for sylvain.poulain@ird.fr
    Look for email sylvain.poulain@ird.fr in Contact file
      Add orcid
      Add First and Last Name
      Add Affiliation
  Looking for ainaledon294@gmail.com
    Look for email ainaledon294@gmail.com in Contact file
      Add orcid
      Add First and Last Name
      Add Affiliation
  Looking for a.herinomena11@gmail.com
    Look for email a.herinomena11@gmail.com in Contact file
      Add First and Last Name
      Add Affiliation
  Looking for md.nodc@ihsm.mg
    Look for email md.nodc@ihsm.mg in Contact file
      Add First and Last Name
      Add Affiliation
  Looking for julien.barde@ird.fr
    Look for email julien.barde@ird.fr in Contact file
      Add orcid
      Add First and Last Name
      Add Affiliation
{'metadata': {'title': 'Images drone UAV, Nosy ve, Madagascar - 20230430 - 01_6', 'publication_date': '2023-